# YOLO

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
model = YOLO("https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8n.pt")
results = model("/content/Parstech-ai.jpg")
for result in results:
    result.save(filename="result.jpg")


100%|██████████| 6.25M/6.25M [00:00<00:00, 111MB/s]



image 1/1 /content/Parstech-ai.jpg: 480x640 8 persons, 1 handbag, 1 tie, 44.5ms
Speed: 35.3ms preprocess, 44.5ms inference, 837.0ms postprocess per image at shape (1, 3, 480, 640)


In [ ]:
model.export(format="onnx")

In [ ]:
onnx_model = YOLO("/content/weights/yolov8n.onnx")
results = onnx_model("/content/Parstech-ai.jpg")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading /content/weights/yolov8n.onnx for ONNX Runtime inference...
requirements: Ultralytics requirement ['onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.1/223.1 MB 246.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 202.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 314.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 13.2s, installed 1 package: ['onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


image 1/1 /content/Parstech-ai.jpg: 640x640 7 persons, 14.9ms
Speed: 2.3ms preprocess, 14.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


# sklearn

In [ ]:
!pip install skl2onnx

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
X,y = iris.data , iris.target
X = X.astype(np.float32)
X_train , X_test,y_train,y_test = train_test_split(X,y)
model = RandomForestClassifier()
model.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
import skl2onnx
onnx_model = skl2onnx.to_onnx(model , X[0])
with open("skl2onnxmodel.onnx","wb") as f:
    f.write(onnx_model.SerializeToString())

# torch

In [ ]:
!pip install onnx
!pip install onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 665.7/665.7 kB 13.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
torch_model = MyModel()
torch_input = torch.randn(1, 1, 32, 32)
onnx_model = torch.onnx.dynamo_export(torch_model, torch_input)

In [ ]:
onnx_model.save("my_image_classifier.onnx")

# TensorFlow

In [ ]:
!pip install tf2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 8.5 MB/s eta 0:00:00


In [25]:
import numpy as np
import tensorflow as tf
import onnx
from tensorflow.keras.applications.resnet50 import ResNet50
model = ResNet50(weights="imagenet",include_top=True)


In [26]:
import tf2onnx
onnx_model,_ = tf2onnx.convert.from_keras(model,
    [tf.TensorSpec(
        model.inputs[0].shape,
        dtype = model.inputs[0].dtype,
        name = model.inputs[0].name
    ) ])

In [27]:
onnx.save(onnx_model,"ResNet50.onnx")